一个现代AI Agent系统中“工具”的理想形态：不是孤立的函数，而是能够感知环境、保持状态、利用记忆、实时交互的智能组件。ToolRuntime是实现这一切的关键桥梁，它将工具与系统级资源连接起来，具有以下优点：
1. 解耦与复用：工具不再需要硬编码数据库连接或用户信息，而是通过统一接口访问资源，提高可维护性和可移植性。
2. 智能化增强：工具不再是“无状态”、“无记忆”的简单函数，而是能感知上下文、记住历史、个性化响应的“智能代理”。
3. 支持复杂场景：如客服机器人、个性化推荐、多轮任务处理等都需要这种运行时上下文能力。
4. 便于测试与调试：所有资源都可通过配置注入，方便单元测试和模拟运行。

工具可以通过 ToolRuntime 参数访问运行时信息，该参数提供以下内容：
- 状态（State）：在执行过程中传递的可变数据（例如消息、计数器、自定义字段等）
- 上下文（Context）：不可变的配置信息，如用户 ID、会话详情或应用特定的配置
- 存储（Store）：跨对话持久化的长期记忆
- 流写入器（Stream Writer）：在工具执行过程中流式发送自定义更新
- 配置（Config）：当前执行的 RunnableConfig
- 工具调用 ID（Tool Call ID）：当前工具调用的唯一标识符

<div align=center><img src="https://techinsight.com.cn/course/Snipaste_2026-02-06_22-21-56.png" width=80%></div>

在LangGraph Pregel对象运行的过程中，会将invoke、stream等图运行方法中state、config、context等参数动态绑定到Runtime对象上，而后ToolRuntime会自行打包这些信息：

 > 源码位置：langgraph/pregel/main.py

In [ ]:
tool_runtime = ToolRuntime(
    state=state,  # 从 input 中提取
    tool_call_id=call["id"],  # 从 tool call 中获取
    config=cfg,  # 从 config_list 中获取
    context=runtime.context,  # 从 Runtime 中获取
    store=runtime.store,  # 从 Runtime 中获取
    stream_writer=runtime.stream_writer,  # 从 Runtime 中获取
)

只需要在工具函数中添加 ToolRuntime 参数，就可以访问运行时信息，并且这个参数是不会暴露给模型的。

In [15]:
from langchain_deepseek import ChatDeepSeek

model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

# 访问state

## 访问当前的对话状态

In [ ]:
from langchain.messages import HumanMessage, ToolMessage
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState

@tool
def weather_search(
    location: str,
    runtime: ToolRuntime  # ToolRuntime 参数对模型不可见
) -> str:
    """联网查询天气"""
    messages = runtime.state["messages"]

    print(f"用户问题是：{messages[0].content}\n识别到查询的城市是：{location}")

    return {
        "messages":[ToolMessage(content=f"{location}天气是晴天", tool_call_id=runtime.tool_call_id)]
    }

agent = create_agent(
    model=model,
    system_prompt="你是一个乐于助人的助手",
    tools=[weather_search],
    state_schema=AgentState
)

result = agent.invoke({"messages": [HumanMessage(content="宁波天气如何")]})
print(result)

## 访问自定义状态字段

In [ ]:
@tool
def get_user_preference(
    pref_name: str,
    runtime: ToolRuntime
) -> str:
    """获取用户偏好设置的值。"""
    preferences = runtime.state.get("user_preferences", {})
    return preferences.get(pref_name, "未设置")

# 更新State

通过Command和updata的组合，我们可以很轻松地在工具中更新state中的值。

## RemoveMessage

RemoveMessage是一个特殊的消息类型，可以在state中移除指定id的消息，搭配update和特殊的变量REMOVE_ALL_MESSAGES可以移除所有消息。


In [ ]:
from langchain.messages import RemoveMessage
from langgraph.types import Command
from langgraph.graph.message import REMOVE_ALL_MESSAGES
from langchain.tools import tool, ToolRuntime

# 通过RemoveMessage清空历史消息
@tool
def clear_conversation() -> Command:
    """Clear the conversation history."""

    return Command(
        update={
        # state中messages是一个list[BaseMessage]，所以这里传入的也是一个列表，列表中支持放置多个删除指令
            "messages": [RemoveMessage(id=REMOVE_ALL_MESSAGES)],
        }
    )

# 在工具中更新自定义状态键
@tool
def update_user_name(
    new_name: str,
    runtime: ToolRuntime
) -> Command:
    """Update the user's name."""
    return Command(update={"user_name": new_name})

清空消息的操作通常发生在上下文的压缩操作中，通过中间件装饰器来实现，这里不做演示，后面课程会详细进行介绍。

# 访问上下文

在调用agent时传递context参数，就可以在工具调用时，通过ToolRuntime对象获取context信息。

In [19]:
from dataclasses import dataclass
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# 模拟用户数据库
USER_DATABASE = {
    "user123": {
        "name": "Alice Johnson",
        "account_type": "Premium",
        "balance": 5000,
        "email": "alice@example.com"
    },
    "user456": {
        "name": "Bob Smith",
        "account_type": "Standard",
        "balance": 1200,
        "email": "bob@example.com"
    }
}

# 定义用户上下文数据结构
@dataclass
class UserContext:
    user_id: str

# 定义工具函数
@tool
def get_account_info(runtime: ToolRuntime[UserContext]) -> str:
    """获取当前用户的账户信息。"""
    # 从运行时上下文中提取用户ID
    user_id = runtime.context.user_id

    # 根据用户ID查询模拟数据库
    if user_id in USER_DATABASE:
        user = USER_DATABASE[user_id]
        return f"账户持有人: {user['name']}\n账户类型: {user['account_type']}\n余额: ${user['balance']}"
    return "未找到用户"

# 创建智能体
agent = create_agent(
    model,
    tools=[get_account_info],
    context_schema=UserContext, # 指定上下文模式
    system_prompt="你是一个财务助理。"
)

# 调用智能体
result = agent.invoke(
    {"messages": [{"role": "user", "content": "我当前的余额是多少？"}]},
    context=UserContext(user_id="user123") # 传入用户上下文
)
for msg in result["messages"]:
    msg.pretty_print()

================================ Human Message =================================

我当前的余额是多少？
================================== Ai Message ==================================

我来帮您查询当前的账户余额信息。
Tool Calls:
  get_account_info (call_00_tl62WlwbiO1uhAG2A5JbkVWJ)
 Call ID: call_00_tl62WlwbiO1uhAG2A5JbkVWJ
  Args:
================================= Tool Message =================================
Name: get_account_info

账户持有人: Alice Johnson
账户类型: Premium
余额: $5000
================================== Ai Message ==================================

根据查询结果，您当前的账户信息如下：

**账户持有人：** Alice Johnson  
**账户类型：** Premium  
**当前余额：** $5,000.00

您的账户余额为5,000美元。


# 同时约束上下文类型和状态类型

在 LangGraph / LangChain 的设计中，ToolRuntime 通常包含：
- content：不可变配置（如用户 ID、租户信息），由泛型参数 T 决定类型
- state：可变状态（如消息列表、任务进度），通常是 dict 或自定义的 AgentState

通过`ToolRuntime[ContextType, StateType]`的写法，能够支持同时对上下文类型和状态类型进行约束。

如果现在工具内部再进行约束，可以通过cast来实现：

In [27]:
from typing import cast
@tool
def my_tool(runtime: ToolRuntime[UserContext]) -> str:
    """这是一个工具"""
    state = cast(AgentState, runtime.state)  # 告诉类型检查器 state 是 AgentState
    name = state["user_name"]  # ✅ 现在有类型提示和 key 检查

✅ 优先创建一个统一的 RuntimeContext 类，将所有运行时不可变配置整合成复合上下文，比如将所有上下文信息合并到一个类里面，或者利用嵌套，像下面的代码一样：

In [ ]:
from pydantic import BaseModel

class RuntimeContext(BaseModel):
    user: UserContext
    app: AppContext
    tenant: TenantContext  # 未来可轻松扩展

<span style="color: red;">不建议使用Union[CtxA, CtxB]，这样得在工具函数中手动判断，不安全且容易出错。</span>

这块内容涉及到store的增删改查，后续会单独进行讲解，这里仅演示通过ToolRuntime对象访问store。

In [ ]:
from typing import Any
from langgraph.store.memory import InMemoryStore
from langchain.agents import create_agent
from langchain.tools import tool, ToolRuntime


# 访问记忆存储
@tool
def get_user_info(user_id: str, runtime: ToolRuntime) -> str:
    """查询用户信息。"""
    store = runtime.store
    user_info = store.get(("users",), user_id)
    return str(user_info.value) if user_info else "未知用户"

# 更新记忆存储
@tool
def save_user_info(user_id: str, user_info: dict[str, Any], runtime: ToolRuntime) -> str:
    """保存用户信息。"""
    store = runtime.store
    store.put(("users",), user_id, user_info)
    return "成功保存用户信息。"

# 初始化内存存储器
store = InMemoryStore()

# 创建智能体，并绑定存储
agent = create_agent(
    model,
    tools=[get_user_info, save_user_info],
    store=store
)

# 第一次会话：保存用户信息
agent.invoke({
    "messages": [{"role": "user", "content": "请保存以下用户信息：用户ID: abc123, 姓名: Foo, 年龄: 25, 邮箱: foo@langchain.dev"}]
})


In [ ]:
# 第二次会话：获取用户信息
agent.invoke({
    "messages": [{"role": "user", "content": "查询 ID 为 'abc123' 的用户信息"}]
})

# 访问stream_writer

stream_writer作用是自定义流式输出的内容，在流式输出章节，会具体介绍。ToolRuntime同样会识别stream_writer，供工具调用并返回自定义的流式工具输出。

In [25]:
from langchain.agents import create_agent
from langgraph.config import get_stream_writer  
from langchain_deepseek import ChatDeepSeek
from langchain.tools import tool, ToolRuntime

# 初始化 DeepSeek 模型
model = ChatDeepSeek(
    model="deepseek-chat",
    temperature=0.7
)

def get_weather(city: str, runtime: ToolRuntime) -> str:
    """获取指定城市的天气。"""
    # 获取流式写入器
    writer = runtime.stream_writer

    # 直接实例化一个writer对象也可以，但不推荐，不是标准做法
    # writer = get_stream_writer()
    
    # 流式传输中间步骤信息
    writer(f"正在查询城市数据: {city}")
    writer(f"已获取城市数据: {city}")
    
    return f"{city} 总是阳光明媚！"

# 创建智能体
agent = create_agent(
    model=model,
    tools=[get_weather],
)

# 执行流式调用
for chunk in agent.stream(
    {"messages": [{"role": "user", "content": "宁波的天气怎么样？"}]},
    stream_writer = get_stream_writer,
    stream_mode="custom"  # 使用自定义流模式以捕获 writer 发送的数据
):
    print(chunk)

通过runtim导入和直接实例writer对象的区别：

| 特性 | `runtime.stream_writer` | 直接实例化/全局导入 |
| :--- | :--- | :--- |
| 上下文感知 | 强（自动绑定当前请求/会话） | 弱（通常需要手动传入上下文） |
| 多用户安全 | 安全（自动隔离不同用户流） | 危险（容易导致数据串流） |
| 代码规范 | 推荐（符合框架设计模式） | 不推荐（破坏解耦） |
| 灵活性 | 高（框架可动态控制 writer 行为） | 低（写死在代码中） |

# 工具和节点的对比

从“执行代码”这个底层动作上看，工具和节点确实很像，都是在运行Python逻辑。但从“工作流编排”的架构角色上看，它们处于不同的层级，职责划分得很清楚：

LangGraph 节点 = 部门经理
- 职责：负责管理一个部门（比如销售部、财务部）。他决定部门内怎么干活，或者把任务分派给手下的员工。
- 特点：层级较高，负责流程控制和状态管理。

工具 = 具体办事的员工 / 专业技能
- 职责：负责做一件非常具体的事（比如“查天气”、“发邮件”、“算加法”）。他听从经理的调遣。
- 特点：功能单一，高内聚，通常被节点调用。

得益于ToolRuntime的引入，现在工具也具有了上下文的感知能力，并且ToolRuntime是包含state属性的，而节点Runtime是不包含state属性的，因为state已经通过单独的参数进行了传递，这点需要注意。